<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pathlib

In [2]:
file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

sheets_info = {
    'n_uni': {'skiprows': 2, 'cols': ['uni']},
    'n_proj': {'skiprows': 3, 'cols': ['proj', 'nome_proj']},
    'n_elem': {'skiprows': 3, 'cols': ['elem', 'nome_elem']},
    'n_vinc': {'skiprows': 3, 'cols': ['vinc', 'nome_vinc']},
    'n_rub': {'skiprows': 3, 'cols': ['rub', 'nome_rub']}
}

dfs = {
    sheet: pd.read_excel(file_path, sheet_name=sheet, skiprows=info['skiprows'], header=None, names=info['cols'])
    for sheet, info in sheets_info.items()
}

for sheet, df in dfs.items():
  if 'uni' in df.columns:
      df['nome_uni'] = df['uni'].str[5:]
      df['uni_int'] = df['uni'].str[:4].astype(int)
  if 'proj' in df.columns:
      df['proj_int'] = df['proj'].str[:4].astype(int)
  if 'elem' in df.columns:
      df['elem_int'] = df['elem'].str[:6].astype(int)
  if 'rub' in df.columns:
      df['rub_int'] = df['rub'].str[:12].astype('int64')
  if 'vinc' in df.columns:
      df['vinc_int'] = df['vinc'].str[:4].astype(int)

df_uni, df_proj, df_elem, df_vinc, df_rub = dfs.values()

In [ ]:
# Generating a dataframe with paid expenses data from 2002 to 2023

# Define the folder path
exp_folder_2002_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2002_2023')

# Read all excel files efficiently
df_list = [
    pd.read_excel(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2002_2023.glob("*.xls")
]

# Concatenate all DataFrames
df_exp_2002_2023 = pd.concat(df_list, ignore_index=True)

# Rename and clean up columns
df_exp_2002_2023 = df_exp_2002_2023[['CPTPAG', 'RESUL_PAGO', 'UNIDORC', 'PROJATIV', 'RUBRICA', 'VINCORC']].rename(columns={
    'CPTPAG':'Comp.pagto.', 'RESUL_PAGO':'Result. pago', 'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ',
    'RUBRICA':'Rubrica', 'VINCORC':'Vinc. Orçam.'
})

# Filter out rows with nan values in some columns
df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.', 'Result. pago'], ignore_index=True)

# Process categorical column efficiently
df_exp_2002_2023['Elemento'] = df_exp_2002_2023['Rubrica'].astype(str).str[:6].astype(int)

In [ ]:
# Generating a dataframe with paid expenses data from 2024 onwards

# Define the folder path
exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

# Read all CSV files efficiently
df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.glob("*.csv")
]

# Concatenate all DataFrames
df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

# Convert financial columns efficiently
cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
df_exp_2024_onwards[cols_to_convert] = df_exp_2024_onwards[cols_to_convert].apply(
    lambda col: pd.to_numeric(col.str.replace('R\$ ', '', regex=True).str.replace(',', '.'), errors='coerce')
)

# Filter out rows with zero values in all columns to convert
df_exp_2024_onwards = df_exp_2024_onwards[
    df_exp_2024_onwards[cols_to_convert].ne(0).any(axis=1)
]

# Rename and clean up columns
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
}, inplace=True)

# Fix month values
df_exp_2024_onwards['MONTH'] = df_exp_2024_onwards['MONTH'].replace({0: 1, 13: 12}).astype(int)

# Convert date
df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

# Process categorical columns efficiently
df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)

# Sorting once before processing
df_exp_2024_onwards.sort_values('Comp.pagto.', inplace=True, ignore_index=True)

# Subtract the last nonzero appearance for target columns
cols = ['Proj/Ativ', 'Rubrica', 'fonterecursos', 'YEAR']
target_cols = ['restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
for target_col in target_cols:
    df_exp_2024_onwards[target_col + '_prev'] = (
        df_exp_2024_onwards.where(df_exp_2024_onwards[target_col] != 0).groupby(cols)[target_col].shift(1).fillna(0)
    )
    df_exp_2024_onwards[target_col] -= df_exp_2024_onwards[target_col + '_prev']

# Compute final results
df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1)

# Final filter and reordering columns
df_exp_2024_onwards = df_exp_2024_onwards[df_exp_2024_onwards['Result. pago'] != 0].reset_index(drop=True)
df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)

df_exp_2024_onwards = df_exp_2024_onwards[
    ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
     'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos']
]

In [ ]:
# concatenating the expense dataframes
relevant_columns = ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
df_exp = pd.concat([df_exp_2002_2023, df_exp_2024_onwards[relevant_columns]], ignore_index=True)

In [ ]:
# Updating the elements dataframe

df_new_elem = df_exp_2024_onwards[['Elemento', 'deelemento']].drop_duplicates(keep='last').rename(columns={
    'Elemento': 'elem_int',
    'deelemento': 'nome_elem'
})
df_elem = pd.concat([df_elem, df_new_elem], ignore_index=True).drop_duplicates(subset='elem_int', keep='last').reset_index(drop=True)
df_elem['elem'] = df_elem['elem_int'].astype(str) + ' - ' + df_elem['nome_elem'].str.title()

In [ ]:
# Updating the rubric dataframe

df_new_rub = df_exp_2024_onwards[['Rubrica', 'dedesdobramento']].drop_duplicates(keep='last').rename(columns={
    'Rubrica': 'rub_int',
    'dedesdobramento': 'nome_rub'
})
df_rub = pd.concat([df_rub, df_new_rub], ignore_index=True).drop_duplicates(subset='rub_int', keep='last').reset_index(drop=True)
df_rub['rub'] = df_rub['rub_int'].astype(str) + ' - ' + df_rub['nome_rub'].str.title()

In [ ]:
# Updating the project dataframe

df_new_proj = df_exp_2024_onwards[['Proj/Ativ', 'desubacao']].drop_duplicates(keep='last').rename(columns={
    'Proj/Ativ': 'proj_int',
    'desubacao': 'nome_proj'
})
df_proj = pd.concat([df_proj, df_new_proj], ignore_index=True).drop_duplicates(subset='proj_int', keep='last').reset_index(drop=True)
df_proj['proj'] = df_proj['proj_int'].astype(str) + ' - ' + df_proj['nome_proj'].str.title()

In [ ]:
# Updating the budget units dataframe

df_new_uni = df_exp_2024_onwards[['Unid. Orçam.', 'deunidadeorcamentária']].drop_duplicates(keep='last').rename(columns={
    'Unid. Orçam.': 'uni_int',
    'deunidadeorcamentária': 'nome_uni'
})
df_uni = pd.concat([df_uni, df_new_uni], ignore_index=True).drop_duplicates(subset='uni_int', keep='last').reset_index(drop=True)
df_uni['uni'] = df_uni['uni_int'].astype(str) + ' - ' + df_uni['nome_uni'].str.title()

In [ ]:
# Updating the entail dataframe

df_new_vinc = df_exp_2024_onwards[['Vinc. Orçam.', 'defonterecursos']].drop_duplicates(keep='last').rename(columns={
    'Vinc. Orçam.': 'vinc_int', 'defonterecursos': 'nome_vinc'
})
df_new_vinc['nome_vinc'] = df_new_vinc['nome_vinc'].str.split('-').str.get(1).str.strip()
df_vinc = pd.concat([df_vinc, df_new_vinc], ignore_index=True).drop_duplicates(subset='vinc_int', keep='last').reset_index(drop=True)
df_vinc['vinc'] = df_vinc['vinc_int'].astype(str) + ' - ' + df_vinc['nome_vinc'].str.title()

In [ ]:
final_data_folder = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/final_data')

df_uni.sort_values('uni_int').to_csv(final_data_folder / 'df_uni.csv', encoding='utf-8-sig', sep=';', index=False)
df_proj.sort_values('proj_int').to_csv(final_data_folder / 'df_proj.csv', encoding='utf-8-sig', sep=';', index=False)
df_elem.sort_values('elem_int').to_csv(final_data_folder / 'df_elem.csv', encoding='utf-8-sig', sep=';', index=False)
df_rub.sort_values('rub_int').to_csv(final_data_folder / 'df_rub.csv', encoding='utf-8-sig', sep=';', index=False)
df_vinc.sort_values('vinc_int').to_csv(final_data_folder / 'df_vinc.csv', encoding='utf-8-sig', sep=';', index=False)
df_exp.sort_values('Comp.pagto.').to_csv(final_data_folder / 'df_exp.csv', encoding='utf-8-sig', sep=';', index=False)

**Revenue**

In [116]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.drop('Valor Orçado', axis=1, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                                 'Mês': 'MONTH',
                                 'Rótulos de Linha': 'orgao',
                                 'Vínculo Cod': 'vinculo',
                                 'N6 Subalinea': 'desdobramento6',
                                 'Valor Arrecadado': 'valor_arrecadado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
df_rev_2004_2017 = df_rev_2004_2017.loc[df_rev_2004_2017['valor_arrecadado'] != 0].reset_index(drop=True)
df_rev_2004_2017.sort_values('Data', inplace=True, ignore_index=True)

df_rev_from_to_2004_2017 = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/rev_from_to-2004_2017.xlsx')
df_rev_from_to_2004_2017['rubrica'] = df_rev_from_to_2004_2017['rubricacompleta'].str.extract(r'^(\d+)')
df_rev_from_to_2004_2017["nome_rubrica"] = df_rev_from_to_2004_2017["rubricacompleta"].str.extract(r"^\d+\s*-\s*(.*)")

df_rev_2004_2017 = df_rev_2004_2017.merge(
    df_rev_from_to_2004_2017[['vinculo', 'desdobramento6', 'rubrica', 'nome_rubrica']],
    on=['vinculo', 'desdobramento6'], how='left'
)

In [117]:
# Generating a dataframe with revenue collected from 2018 to 2023

rev_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_list = [
    pd.read_excel(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2018_2023.glob("*.xlsx")
]

df_rev_2018_2023 = pd.concat(df_list, ignore_index=True)

df_rev_2018_2023.drop(columns=[
    'valor_cancelado', 'valor_orcado', 'valor_lancado', 'valor_meta', 'informacao_complementar',
    'nome_orgao', 'orgao', 'nome_orgao.1', 'digito', 'categoria', 'especie', 'desdobramento1',
    'desdobramento2', 'desdobramento3', 'desdobramento4', 'desdobramento5'
    ], inplace=True)
df_rev_2018_2023 = df_rev_2018_2023.loc[(df_rev_2018_2023['orgao_raiz'] == 7000) & (df_rev_2018_2023['valor_arrecadado'] != 0)].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023.sort_values('Data', inplace=True, ignore_index=True)

df_rev_from_to_2018_2023 = pd.read_excel(r'/content/drive/MyDrive/Dashboard_data/utils/rev_from_to-2018_2023.xlsx')
df_rev_from_to_2018_2023['rubrica_new'] = df_rev_from_to_2018_2023['rubricacompleta'].str.extract(r'^(\d+)')
df_rev_from_to_2018_2023["nome_rubrica"] = df_rev_from_to_2018_2023["rubricacompleta"].str.extract(r"^\d+\s*-\s*(.*)")

df_rev_2018_2023 = df_rev_2018_2023.merge(
    df_rev_from_to_2018_2023.drop('rubricacompleta', axis=1), on='rubrica', how='left'
    ).drop('rubrica', axis=1).rename(columns={'rubrica_new': 'rubrica'})

In [118]:
# Generating a dataframe with revenue collected from 2024 onwards

rev_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2024+')

df_list = [
    pd.read_csv(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2024_onwards.glob("*.csv")
]

df_rev_2024_onwards = pd.concat(df_list, ignore_index=True)

df_rev_2024_onwards.rename(columns={'exercicio': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2024_onwards['Data'] = pd.to_datetime(df_rev_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2024_onwards.sort_values('Data', inplace=True, ignore_index=True)

df_otr = df_rev_2024_onwards[['origem', 'tipo', 'rubricacompleta']].drop_duplicates(keep='last').rename(
    columns={'origem': 'o', 'tipo': 't', 'rubricacompleta': 'r'}
)

for column in df_otr:
  df_otr[column + '_int'] = df_otr[column].str.extract(r'^(\d+)')
  df_otr['nome_' + column] = df_otr[column].str.extract(r"^\d+\s*-\s*(.*)")

In [119]:
df = pd.concat([df_rev_2004_2017, df_rev_2018_2023], ignore_index=True)
df

,orgao,vinculo,desdobramento6,YEAR,MONTH,valor_arrecadado,Data,rubrica,nome_rubrica,orgao_raiz,origem,tipo
0,7000.0,400,12102999 - Outras Contribuições Previdenciárias,2004,1,37.56,2004-01-01,12190000000000,Outras Contribuições Sociais,NaN,NaN,NaN
1,7000.0,6050,72102901 - Contribuição Patronal de Servidor A...,2004,1,179881.44,2004-01-01,72180310000000,Contr. Patronal - Servidor Ativo - Plano em Ca...,NaN,NaN,NaN
2,7000.0,400,19909900 - Outras Receitas,2004,1,4699.39,2004-01-01,1999992118,Outras Receitas - Diversas - Adm. Indireta,NaN,NaN,NaN
3,7000.0,400,72102901 - Contribuição Patronal de Servidor A...,2004,1,2665961.14,2004-01-01,72180310000000,Contr. Patronal - Servidor Ativo - Plano em Re...,NaN,NaN,NaN
4,7000.0,400,12102907 - Contribuição de Servidor - Ativo Civil,2004,1,2917709.38,2004-01-01,12180110000000,Contr.do Servidor Civil Ativo - Plano em Repar...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11490,NaN,6049,12150121010141 - Contr.do Servidor Civil Inati...,2023,12,1521771.33,2023-12-01,1215012141,Contr.do Servidor Civil Inativo - Plano em Rep...,7000.0,12 - Contribuições,12150121 - Contr.do Servidor Civil Inativo - P...
11491,NaN,6049,12150121010151 - Contr.do Servidor Civil Inati...,2023,12,473345.30,2023-12-01,1215012151,Contr.do Servidor Civil Inativo - Plano em Rep...,7000.0,12 - Contribuições,12150121 - Contr.do Servidor Civil Inativo - P...
11492,NaN,6049,12150121010161 - Contr.do Servidor Civil Inati...,2023,12,121083.62,2023-12-01,1215012161,Contr.do Servidor Civil Inativo - Plano em Rep...,7000.0,12 - Contribuições,12150121 - Contr.do Servidor Civil Inativo - P...
11493,NaN,6050,72150211020111 - Contr. Patronal - Serv. Ativo...,2023,12,29637700.08,2023-12-01,7215021112,Contr.Patronal - Servidor Ativo - Plano em Cap...,7000.0,72 - Contribuições Intra-Orçamentárias,72150211 - Contr. Patronal - Servidor Civil At...


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11495 entries, 0 to 11494
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   orgao             3264 non-null   float64       
 1   vinculo           11495 non-null  int64         
 2   desdobramento6    11495 non-null  object        
 3   YEAR              11495 non-null  int64         
 4   MONTH             11495 non-null  int64         
 5   valor_arrecadado  11495 non-null  float64       
 6   Data              11495 non-null  datetime64[ns]
 7   rubrica           11495 non-null  object        
 8   nome_rubrica      11495 non-null  object        
 9   orgao_raiz        8231 non-null   float64       
 10  origem            8231 non-null   object        
 11  tipo              8231 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 1.1+ MB


In [127]:
df[['nome_rubrica']].drop_duplicates()

,nome_rubrica
0,Outras Contribuições Sociais
1,Contr. Patronal - Servidor Ativo - Plano em Ca...
2,Outras Receitas - Diversas - Adm. Indireta
3,Contr. Patronal - Servidor Ativo - Plano em Re...
4,Contr.do Servidor Civil Ativo - Plano em Repar...
...,...
10426,Parc.de Débitos-Patronal-LC nº 637/2010-Centra...
10441,Comp. Finan. RPPS/RPPS - Estados - Plano em Re...
10482,Contr.do Servidor Civil Inativo - Plano em Cap...
10643,Aluguéis Diversos - Taxa de Administração - IN...


In [124]:
df.columns

Index(['orgao', 'vinculo', 'desdobramento6', 'YEAR', 'MONTH',
       'valor_arrecadado', 'Data', 'rubrica', 'nome_rubrica', 'orgao_raiz',
       'origem', 'tipo'],
      dtype='object')

In [122]:
df_otr.columns

Index(['o', 't', 'r', 'o_int', 'nome_o', 't_int', 'nome_t', 'r_int', 'nome_r'], dtype='object')

In [123]:
df.merge(df_otr, left_on='rubrica', right_on='r_int', how='left')#.to_excel('df.xlsx')

,orgao,vinculo,desdobramento6,YEAR,MONTH,valor_arrecadado,Data,rubrica,nome_rubrica,orgao_raiz,...,tipo,o,t,r,o_int,nome_o,t_int,nome_t,r_int,nome_r
0,7000.0,400,12102999 - Outras Contribuições Previdenciárias,2004,1,37.56,2004-01-01,12190000000000,Outras Contribuições Sociais,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000.0,6050,72102901 - Contribuição Patronal de Servidor A...,2004,1,179881.44,2004-01-01,72180310000000,Contr. Patronal - Servidor Ativo - Plano em Ca...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7000.0,400,19909900 - Outras Receitas,2004,1,4699.39,2004-01-01,1999992118,Outras Receitas - Diversas - Adm. Indireta,NaN,...,NaN,19 - Outras Receitas Correntes,19999921 - Outras Receitas Não Arrec.e Não Pro...,1999992118 - Outras Receitas - Diversas - Adm....,19,Outras Receitas Correntes,19999921,Outras Receitas Não Arrec.e Não Projetadas p/ ...,1999992118,Outras Receitas - Diversas - Adm. Indireta
3,7000.0,400,72102901 - Contribuição Patronal de Servidor A...,2004,1,2665961.14,2004-01-01,72180310000000,Contr. Patronal - Servidor Ativo - Plano em Re...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7000.0,400,12102907 - Contribuição de Servidor - Ativo Civil,2004,1,2917709.38,2004-01-01,12180110000000,Contr.do Servidor Civil Ativo - Plano em Repar...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11490,NaN,6049,12150121010141 - Contr.do Servidor Civil Inati...,2023,12,1521771.33,2023-12-01,1215012141,Contr.do Servidor Civil Inativo - Plano em Rep...,7000.0,...,12150121 - Contr.do Servidor Civil Inativo - P...,12 - Contribuições,12150121 - Contr.do Servidor Civil Inativo - P...,1215012141 - Contr.do Servidor Civil Inativo -...,12,Contribuições,12150121,Contr.do Servidor Civil Inativo - Principal,1215012141,Contr.do Servidor Civil Inativo - Plano em Rep...
11491,NaN,6049,12150121010151 - Contr.do Servidor Civil Inati...,2023,12,473345.30,2023-12-01,1215012151,Contr.do Servidor Civil Inativo - Plano em Rep...,7000.0,...,12150121 - Contr.do Servidor Civil Inativo - P...,12 - Contribuições,12150121 - Contr.do Servidor Civil Inativo - P...,1215012151 - Contr.do Servidor Civil Inativo -...,12,Contribuições,12150121,Contr.do Servidor Civil Inativo - Principal,1215012151,Contr.do Servidor Civil Inativo - Plano em Rep...
11492,NaN,6049,12150121010161 - Contr.do Servidor Civil Inati...,2023,12,121083.62,2023-12-01,1215012161,Contr.do Servidor Civil Inativo - Plano em Rep...,7000.0,...,12150121 - Contr.do Servidor Civil Inativo - P...,12 - Contribuições,12150121 - Contr.do Servidor Civil Inativo - P...,1215012161 - Contr.do Servidor Civil Inativo -...,12,Contribuições,12150121,Contr.do Servidor Civil Inativo - Principal,1215012161,Contr.do Servidor Civil Inativo - Plano em Rep...
11493,NaN,6050,72150211020111 - Contr. Patronal - Serv. Ativo...,2023,12,29637700.08,2023-12-01,7215021112,Contr.Patronal - Servidor Ativo - Plano em Cap...,7000.0,...,72150211 - Contr. Patronal - Servidor Civil At...,72 - Contribuições Intra-Orçamentárias,72150211 - Contr. Patronal - Servidor Civil At...,7215021112 - Contr.Patronal - Servidor Ativo -...,72,Contribuições Intra-Orçamentárias,72150211,Contr. Patronal - Servidor Civil Ativo - Princ...,7215021112,Contr.Patronal - Servidor Ativo - Plano em Cap...


In [ ]:
df.loc[df['rubrica']]

In [76]:
df_otr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 44 to 1652
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   origem                145 non-null    object
 1   tipo                  145 non-null    object
 2   rubricacompleta       145 non-null    object
 3   origem_int            145 non-null    object
 4   nome_origem           145 non-null    object
 5   tipo_int              145 non-null    object
 6   nome_tipo             145 non-null    object
 7   rubricacompleta_int   145 non-null    object
 8   nome_rubricacompleta  145 non-null    object
dtypes: object(9)
memory usage: 11.3+ KB


In [37]:
df_rev_2024_onwards.loc[
    (df_rev_2024_onwards['fonterecurso'] == '1802069001 - 6069 - TAXA DE ADMINISTRAÇÃO DO RPPS') &
    (df_rev_2024_onwards['YEAR'] == 2024), 'valorarrecadadobruto'].sum()

33720176.19

In [ ]:
df_ori = df_rev_2024_onwards[['origem']].drop_duplicates(keep='last').reset_index(drop=True)
df_ori['origem_int'] = df_ori['origem'].str[:2].astype(int)
df_ori